# Modules

In [15]:
import pandas as pd

# Variables

In [16]:
# National emissions reported to the Convention on Long-range Transboundary Air Pollution (LRTAP Convention), 2023
# https://sdi.eea.europa.eu/catalogue/srv/eng/catalog.search#/metadata/2999364f-be52-4012-b4fd-f98e2cc8fab6
ds_NatEmissions='01.Data/eea_t_lrtap-convention_p_1990-2021_v01_r00/CLRTAP_NVFR19_V23_1_GF_csv.csv'

In [17]:
# National emissions reported to the UNFCCC and to the EU Greenhouse Gas Monitoring Mechanism, October 2023
# https://sdi.eea.europa.eu/catalogue/srv/eng/catalog.search#/metadata/e2e7dd1e-0d67-4b20-a0d4-b22c53a59d24
ds_UNFCCEmissions='01.Data/eea_t_national-emissions-reported_p_2023_v01_r01/UNFCCC_v26.csv'

# Load data

In [18]:
df_NatEmissions=pd.read_csv(ds_NatEmissions,header=0,delimiter="\t")

/var/folders/n2/fd_16hwd5b58jtjy0v7thtzh0000gn/T/ipykernel_13854/1976418945.py:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_NatEmissions=pd.read_csv(ds_NatEmissions,header=0,delimiter="\t")


In [19]:
df_NatEmissions.head()

,Country_Code,Country,Pollutant_name,Format_name,Sector_code,Year,Emissions,Unit,Notation,VersionId,Parent_sector_code,Sector_name
0,AT,Austria,As,NFR 2014-1,1A1a,1990,NaN,Mg,NaN,1.0,NATIONAL TOTAL,Public electricity and heat production
1,AT,Austria,As,NFR 2014-1,1A1b,1990,NaN,Mg,NaN,1.0,NATIONAL TOTAL,Petroleum refining
2,AT,Austria,As,NFR 2014-1,1A1c,1990,NaN,Mg,NaN,1.0,NATIONAL TOTAL,Manufacture of solid fuels and other energy in...
3,AT,Austria,As,NFR 2014-1,1A2a,1990,NaN,Mg,NaN,1.0,NATIONAL TOTAL,Stationary combustion in manufacturing industr...
4,AT,Austria,As,NFR 2014-1,1A2b,1990,NaN,Mg,NaN,1.0,NATIONAL TOTAL,Stationary combustion in manufacturing industr...


In [24]:
df_NatEmissions.Sector_name.unique()

# 'International aviation LTO (civil)',
# 'Domestic aviation LTO (civil)'

array(['Public electricity and heat production', 'Petroleum refining',
       'Manufacture of solid fuels and other energy industries',
       'Stationary combustion in manufacturing industries and construction: Iron and steel',
       'Stationary combustion in manufacturing industries and construction: Non-ferrous metals',
       'Stationary combustion in manufacturing industries and construction: Chemicals',
       'Stationary combustion in manufacturing industries and construction: Pulp, Paper and Print',
       'Stationary combustion in manufacturing industries and construction: Food processing, beverages and tobacco',
       'Stationary combustion in manufacturing industries and construction: Non-metallic minerals',
       'Mobile Combustion in manufacturing industries and construction',
       'Stationary combustion in manufacturing industries and construction: Other',
       'International aviation LTO (civil)',
       'Domestic aviation LTO (civil)', 'Road transport: Passenger 

In [25]:
df_NatEmissions.Pollutant_name.unique()

array(['As', 'BC', 'benzo(a)', 'benzo(b)', 'benzo(k)', 'Cd', 'CO', 'Cr',
       'Cu', 'dioxin', 'HCB', 'Hg', 'Indeno', 'NH3', 'Ni', 'NMVOC', 'NOx',
       'Pb', 'PCB', 'PM10', 'PM2.5', 'Se', 'SOx', 'total PAH', 'TSP',
       'Zn'], dtype=object)

In [30]:
df_NatEmissions1 = df_NatEmissions[df_NatEmissions.Pollutant_name == 'CO']

In [36]:
#AND
df_NatEmissions2=df_NatEmissions[(df_NatEmissions.Pollutant_name == 'CO') & (df_NatEmissions.Sector_name == 'International aviation LTO (civil)')]
#OR
df_NatEmissions3=df_NatEmissions[(df_NatEmissions.Pollutant_name == 'CO') | (df_NatEmissions.Sector_name == 'International aviation LTO (civil)')]

In [37]:
#df_NatEmissions3.Pollutant_name.unique()
df_NatEmissions3.Sector_name.unique()

array(['International aviation LTO (civil)',
       'Public electricity and heat production', 'Petroleum refining',
       'Manufacture of solid fuels and other energy industries',
       'Stationary combustion in manufacturing industries and construction: Iron and steel',
       'Stationary combustion in manufacturing industries and construction: Non-ferrous metals',
       'Stationary combustion in manufacturing industries and construction: Chemicals',
       'Stationary combustion in manufacturing industries and construction: Pulp, Paper and Print',
       'Stationary combustion in manufacturing industries and construction: Food processing, beverages and tobacco',
       'Stationary combustion in manufacturing industries and construction: Non-metallic minerals',
       'Mobile Combustion in manufacturing industries and construction',
       'Stationary combustion in manufacturing industries and construction: Other',
       'Domestic aviation LTO (civil)', 'Road transport: Passenger 

In [21]:
input_data=pd.read_csv(ds_UNFCCEmissions,header=0)

/var/folders/n2/fd_16hwd5b58jtjy0v7thtzh0000gn/T/ipykernel_13854/794109420.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  input_data=pd.read_csv(ds_UNFCCEmissions,header=0)


In [22]:
input_data.head()

,Country_code,Country,Format_name,Pollutant_name,Sector_code,Sector_name,Parent_sector_code,Unit,Year,emissions,Notation,PublicationDate,DataSource
0,AT,Austria,IPCC Common Reporting Format,All greenhouse gases - (CO2 equivalent),-,- 4(IV) Indirect N2O Emissions from Managed S...,4,Gg CO2 equivalent,1994,12.62568,NaN,20231006,EEA
1,AT,Austria,IPCC Common Reporting Format,All greenhouse gases - (CO2 equivalent),-,- 4(IV) Indirect N2O Emissions from Managed S...,4,Gg CO2 equivalent,1999,11.86116,NaN,20231006,EEA
2,AT,Austria,IPCC Common Reporting Format,All greenhouse gases - (CO2 equivalent),-,- 4(IV) Indirect N2O Emissions from Managed S...,4,Gg CO2 equivalent,2010,12.66384,NaN,20231006,EEA
3,AT,Austria,IPCC Common Reporting Format,All greenhouse gases - (CO2 equivalent),-,- 4(IV) Indirect N2O Emissions from Managed S...,4,Gg CO2 equivalent,2011,12.59482,NaN,20231006,EEA
4,AT,Austria,IPCC Common Reporting Format,All greenhouse gases - (CO2 equivalent),1,1 - Energy,Sectors/Totals_incl_incl,Gg CO2 equivalent,1993,52087.32124,NaN,20231006,EEA


In [23]:
input_data.size

8726068